In [28]:
from openstack import connection
import copy

In [29]:
def connection_to_cluster(project):
    conn = connection.Connection(auth_url="http://controller:35357/v3",
                                 project_name=project,
                                 project_domain_name="default",
                                 user_domain_name="default",
                                 username="admin",
                                 password="QIP@t4Ay")
    return conn

In [34]:
class OSmap(object):
    
    def __init__ (self, connection='', OS=True, with_flavors=True, with_volumes=True, hypervisor_details=False):
        'Инит функция'
        if OS == True:
            self.__conn = connection
            self.__sl = [i for i in self.__conn.compute.servers(all_tenants=True)]
            self.__hl = [i for i in self.__conn.compute.hypervisors()]
            
            if with_flavors == True:
                self.__fd = self.__get_fl_dict()
                self.dict_flavors = copy.deepcopy(self.__fd)
            
            if with_volumes == True:
                self.__vd = self.__get_vl_dict()
                self.dict_volumes = copy.deepcopy(self.__vd)
                
                
            self.dict_hypervisors = self.__get_hp_dict(details=hypervisor_details)
            self.dict_servers = self.__get_vm_dict(flavors=with_flavors)
            self.list_names  = self.__get_names()
    
    
    def get_vm_by_name(self, vmn=''):
        'Функция возвращает объект ВМ на основании имени'
        for i in self.__sl:
            if i.name == vmn:
                return i 
    
    def get_vm_by_id(self, vmn=''):
        'Функция возвращает объект ВМ на основании ID'
        for i in self.__sl:
            if i.id == vmn:
                return i 
    
        
    
    def __get_hp_dict(self, details=False):
        'Функция возвращает словарь гипервизоров'
        d = {}
        for i in self.__hl:
            if details == True:
                d[i.name]=self.__conn.compute.get_hypervisor(i.id).to_dict()
            else:
                d[i.name]=i.to_dict()
        return d   
           
    def __get_fl_dict(self):
        'Функция возвращает словарь флейворов'
        d = {}
        for i in set([i.flavor['id'] for i in self.__sl]):
            d[i]=self.__conn.compute.get_flavor(flavor=i).to_dict()
        return d
    
    def __get_vl_dict(self):
        'Функция возвращает словарь вольюмов'
        d = {}
        for i in [i for i in self.__conn.block_store.volumes(all_tenants=True)]:
            d[i.id]=i.to_dict()
        return d
    
    
    def __get_names (self):
        'Функция возвращает список имен ВМ'
        a = [i.name for i in self.__sl]
        return a
    
            
    def __get_vm_dict(self,flavors=True):
        'Возвращает словарь с описанием ВМ из OS + cpu & ram'
        d = {}
        for i in self.__sl:
            d[i.name] = self.get_vm_by_name(vmn=i.name).to_dict()
            if flavors == True:
                d[i.name]['cpu'] = self.__fd[i.flavor['id']]['vcpus']
                d[i.name]['ram'] = self.__fd[i.flavor['id']]['ram']
        return d
        

In [13]:
%%time
x = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False)

CPU times: user 2.99 s, sys: 92.3 ms, total: 3.09 s
Wall time: 15.8 s


In [14]:
%%time
y = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False)

CPU times: user 3.48 s, sys: 48.2 ms, total: 3.52 s
Wall time: 17.9 s


In [15]:
%%time
z = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True)

CPU times: user 5.16 s, sys: 107 ms, total: 5.26 s
Wall time: 34.7 s


In [16]:
%%time
x1 = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False, hypervisor_details=True)

CPU times: user 4.11 s, sys: 68.5 ms, total: 4.18 s
Wall time: 21.1 s


In [17]:
%%time
y1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False, hypervisor_details=True)

CPU times: user 5.18 s, sys: 31.3 ms, total: 5.21 s
Wall time: 23.8 s


In [36]:
%%time
z1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True, hypervisor_details=True)

CPU times: user 5.98 s, sys: 116 ms, total: 6.09 s
Wall time: 40.4 s


In [31]:
c1 = connection_to_cluster('admin')

In [46]:
z1.dict_volumes

{'e1e312bf-c785-4ad0-987e-acbcc9b24241': {'attachments': [{'attached_at': '2016-12-29T08:44:12.000000',
    'attachment_id': '50b037d7-d348-46ca-bba6-7a5c38a908b2',
    'device': '/dev/vdb',
    'host_name': None,
    'id': 'e1e312bf-c785-4ad0-987e-acbcc9b24241',
    'server_id': '8b793137-bc87-4e11-b989-334273bcf71a',
    'volume_id': 'e1e312bf-c785-4ad0-987e-acbcc9b24241'}],
  'availability_zone': 'nova',
  'consistency_group_id': None,
  'created_at': '2016-12-29T08:43:37.000000',
  'description': None,
  'extended_replication_status': None,
  'host': 's_ctr01t.localdomain@ceph#ceph',
  'id': 'e1e312bf-c785-4ad0-987e-acbcc9b24241',
  'image_id': None,
  'is_bootable': False,
  'is_encrypted': False,
  'links': [{'href': 'http://controller:8776/v2/b83a5fb65edc41a1a745f06aa093d416/volumes/e1e312bf-c785-4ad0-987e-acbcc9b24241',
    'rel': 'self'},
   {'href': 'http://controller:8776/b83a5fb65edc41a1a745f06aa093d416/volumes/e1e312bf-c785-4ad0-987e-acbcc9b24241',
    'rel': 'bookmark'}],

In [62]:
ls = [[z1.dict_volumes[i]['name'],z1.dict_volumes[i]['volume_type']] for i in z1.dict_volumes.keys() if z1.dict_volumes[i]['volume_type'] =='migr-0' ]

In [63]:
len(ls),ls

(3,
 [['is_efem_migr_test_migr-0', 'migr-0'],
  ['100tib_samba_backup', 'migr-0'],
  ['tnx_mon', 'migr-0']])